## Загрузим нужные библиотеки

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.metrics import recall_score, precision_score
from catboost import CatBoostClassifier, cv, Pool

from seed_ import seed_everything
from dicision_stats_features import create_decision

from usr_t_p import create_place_and_score
from tfidf_features import create_tfidf_features

from sklearn.impute import SimpleImputer

%matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

### Fix seed

In [4]:
seed_everything()

In [5]:
df_train = pd.read_csv("train_dataset_train.csv")
df_user = pd.read_csv("user_decision.csv")

### Добавим новых значений

In [6]:
list_id = df_train["id"].unique().tolist()

df_user_train = df_user[df_user["user_id"].isin(list_id)]
df_user_train

,user_id,period,decision_id
0,10625,1,409
2,10775,4,420
3,10236,2,284
4,10130,3,72
5,10273,2,140
...,...,...,...
60220,10528,1,88
60222,10832,3,315
60224,10189,1,33
60226,10424,2,140


In [7]:
new_datas = create_decision(df_user_train)

In [8]:
train = df_train.copy()

for data in new_datas:
    train = train.merge(data, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)


In [9]:
train.shape

(680, 33)

### W2V

In [10]:
# datas = create_w2v_features()

In [11]:
# per1, per2, per3, per4 = datas["period_1"], datas["period_2"], datas["period_3"], datas["period_4"]

In [12]:
# train_w_vec = train.merge(per1, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# train_w_vec = train_w_vec.merge(per2, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# train_w_vec = train_w_vec.merge(per3, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# train_w_vec = train_w_vec.merge(per4, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)

In [13]:
# train_w_vec

### tfidf

In [14]:
tfidf_features = create_tfidf_features()
# a, b, c, d  = create_tfidf_features()

In [15]:
# tmp = a.drop("period", axis=1).merge(b.drop("period", axis=1), how='inner', left_on="user_id", right_on='user_id')
# tmp = tmp.merge(c.drop("period", axis=1), how='inner', left_on="user_id", right_on='user_id')
# tfidf_features = tmp.merge(d.drop("period", axis=1), how='inner', left_on="user_id", right_on='user_id')
# tfidf_features

In [16]:
train_w_vec = train.merge(tfidf_features.reset_index(), how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)

In [17]:
train_w_vec

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,cnt_1,cnt_2,cnt_3,cnt_4,max_1,...,vec_90,vec_91,vec_92,vec_93,vec_94,vec_95,vec_96,vec_97,vec_98,vec_99
0,10884,4.0,4.0,4.0,4.0,7.0,14.0,28.0,10.0,386.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057215,0.000000,0.000000
1,10106,5.0,4.0,5.0,5.0,21.0,21.0,12.0,8.0,333.0,...,0.000000,0.084850,0.084142,0.089202,0.000000,0.000000,0.000000,0.000000,0.000000,0.080402
2,10438,6.0,5.0,5.0,5.0,62.0,45.0,63.0,38.0,424.0,...,0.034689,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,10130,5.0,4.0,5.0,5.0,26.0,28.0,34.0,66.0,410.0,...,0.000000,0.000000,0.000000,0.000000,0.047178,0.046609,0.049347,0.050397,0.036481,0.047719
4,10667,4.0,5.0,5.0,5.0,58.0,70.0,194.0,35.0,412.0,...,0.017048,0.017736,0.000000,0.018645,0.000000,0.000000,0.000000,0.000000,0.000000,0.036145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5.0,4.0,5.0,4.0,10.0,2.0,14.0,4.0,422.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.066079,0.069961,0.000000,0.000000,0.000000
676,10386,5.0,4.0,5.0,5.0,1.0,14.0,24.0,1.0,114.0,...,0.000000,0.000000,0.000000,0.000000,0.056372,0.053363,0.056498,0.057700,0.000000,0.000000
677,10675,5.0,4.0,5.0,4.0,4.0,17.0,23.0,12.0,434.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
678,10366,4.0,4.0,4.0,4.0,10.0,44.0,45.0,38.0,426.0,...,0.045128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### User points

In [18]:
place_score = create_place_and_score()

/Users/19708737/Documents/цифровой_прорыв/MO_2022_2/usr_t_p.py:15: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  score_per_period = pd.DataFrame(data=b.drop("user_id", axis=1).values,
/Users/19708737/Documents/цифровой_прорыв/MO_2022_2/usr_t_p.py:24: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  place_per_period = pd.DataFrame(data=b.drop("user_id", axis=1).values,


In [19]:
train_vec_scores = train_w_vec.merge(place_score, how="left", left_on="id", right_on="user_id")

### Replace NaN

In [20]:
train_vec_scores

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,cnt_1,cnt_2,cnt_3,cnt_4,max_1,...,5_per_2_score,5_per_3_score,5_per_4_score,6_per_0_score,6_per_1_score,6_per_2_score,6_per_3_score,6_per_4_score,game_id,team_id
0,10884,4.0,4.0,4.0,4.0,7.0,14.0,28.0,10.0,386.0,...,16,-8,158,67,26,28,25,4,202,1440
1,10106,5.0,4.0,5.0,5.0,21.0,21.0,12.0,8.0,333.0,...,16,19,158,87,35,29,36,4,211,1536
2,10438,6.0,5.0,5.0,5.0,62.0,45.0,63.0,38.0,424.0,...,16,41,158,55,30,41,49,4,242,1742
3,10130,5.0,4.0,5.0,5.0,26.0,28.0,34.0,66.0,410.0,...,16,-7,158,77,26,29,22,4,184,1272
4,10667,4.0,5.0,5.0,5.0,58.0,70.0,194.0,35.0,412.0,...,16,39,158,55,9,42,50,4,193,1382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5.0,4.0,5.0,4.0,10.0,2.0,14.0,4.0,422.0,...,16,-7,158,59,24,24,27,4,184,1274
676,10386,5.0,4.0,5.0,5.0,1.0,14.0,24.0,1.0,114.0,...,16,3,158,92,27,15,32,4,210,1526
677,10675,5.0,4.0,5.0,4.0,4.0,17.0,23.0,12.0,434.0,...,16,4,158,67,23,21,24,4,235,1670
678,10366,4.0,4.0,4.0,4.0,10.0,44.0,45.0,38.0,426.0,...,16,17,158,77,29,29,38,4,248,1808


In [21]:
# train_vec_scores = train_vec_scores.fillna(0)

In [22]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
tranformed_data = imp_mean.fit_transform(train_vec_scores)

train_vec_scores_nans = pd.DataFrame(tranformed_data, columns=train_vec_scores.columns)

## Выделим выборки

In [23]:
train_vec_scores_nans.to_csv("train_vec_scores_nans.csv", index=False)

In [24]:
targets = ["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"]

In [25]:
X = train_vec_scores_nans.drop(targets, axis = 1).drop(["id", "user_id"], axis = 1).drop(592).drop(544)
y = train_vec_scores_nans.drop(592).drop(544)[targets]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [27]:
y_train_ada = y_train[["Adaptability"]]
y_test_ada = y_test[["Adaptability"]]

y_train_sys = y_train[["Systemic thinking"]]
y_test_sys = y_test[["Systemic thinking"]]

y_train_ana = y_train[["Analytical thinking"]]
y_test_ana = y_test[["Analytical thinking"]]

y_train_foc = y_train[["Focus"]]
y_test_foc = y_test[["Focus"]]

## Обучение модели

In [28]:
params = {"verbose": 250,
          "od_wait": 50,
          "od_type": "Iter",
          "iterations": 2000,
          "depth": 4,
          "learning_rate": 0.007,
          "loss_function": "MultiClass"
          }


model_ada = CatBoostClassifier(**params)
model_sys = CatBoostClassifier(**params)
model_ana = CatBoostClassifier(**params)
model_foc = CatBoostClassifier(**params)

In [29]:
model_ada.fit(X_train, y_train_ada, eval_set=(X_test, y_test_ada))
model_sys.fit(X_train, y_train_sys, eval_set=(X_test, y_test_sys))
model_ana.fit(X_train, y_train_ana, eval_set=(X_test, y_test_ana))
model_foc.fit(X_train, y_train_foc, eval_set=(X_test, y_test_foc))

0:	learn: 1.7820371	test: 1.7826268	best: 1.7826268 (0)	total: 81.7ms	remaining: 2m 43s
250:	learn: 0.9850366	test: 1.0812671	best: 1.0812671 (250)	total: 4.71s	remaining: 32.9s
500:	learn: 0.8463589	test: 1.0028203	best: 1.0028203 (500)	total: 9.07s	remaining: 27.1s
750:	learn: 0.7792510	test: 0.9846834	best: 0.9846225 (748)	total: 14.1s	remaining: 23.5s
1000:	learn: 0.7246170	test: 0.9777319	best: 0.9777319 (1000)	total: 18.6s	remaining: 18.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9772665203
bestIteration = 1058

Shrink model to first 1059 iterations.
0:	learn: 1.5989246	test: 1.5994125	best: 1.5994125 (0)	total: 17.2ms	remaining: 34.5s
250:	learn: 0.8380135	test: 0.9213203	best: 0.9213203 (250)	total: 3.63s	remaining: 25.3s
500:	learn: 0.7293261	test: 0.8543990	best: 0.8543990 (500)	total: 7.03s	remaining: 21s
750:	learn: 0.6743672	test: 0.8343545	best: 0.8342978 (749)	total: 10.6s	remaining: 17.7s
1000:	learn: 0.6291587	test: 0.8242888	best: 0.8242888 

## Оценка точности

In [30]:
pred_ada = model_ada.predict(X_test)
pred_sys = model_sys.predict(X_test)
pred_ana = model_ana.predict(X_test)
pred_foc = model_foc.predict(X_test)

pred = pd.DataFrame(index=X_test.index)
pred["Adaptability"] = pred_ada
pred["Systemic thinking"] = pred_sys
pred["Analytical thinking"] = pred_ana
pred["Focus"] = pred_foc

In [31]:
result = 0 

for i, col in enumerate(y_test.columns):
    result += recall_score(y_test[col], pred[col], average='macro', zero_division=True)

print("Recall score", result/4)

Recall score 0.31552117228535


##local##-##public##
0.30658 - 0.261382
0.30897 - 0.258336
0.31063 - 0.271379 - ans6
0.30201 - 0.263627 - ans7

In [29]:
model_ada = CatBoostClassifier(**params)
model_sys = CatBoostClassifier(**params)
model_ana = CatBoostClassifier(**params)
model_foc = CatBoostClassifier(**params)

model_ada.fit(X, y["Adaptability"])
model_sys.fit(X, y["Systemic thinking"])
model_ana.fit(X, y["Analytical thinking"])
model_foc.fit(X, y["Focus"])

0:	learn: 1.7820186	total: 27.3ms	remaining: 54.6s
250:	learn: 1.0035945	total: 5.4s	remaining: 37.6s
500:	learn: 0.8769046	total: 10.2s	remaining: 30.6s
750:	learn: 0.8182380	total: 15.2s	remaining: 25.3s
1000:	learn: 0.7720311	total: 18.8s	remaining: 18.7s
1250:	learn: 0.7241497	total: 22.2s	remaining: 13.3s
1500:	learn: 0.6792788	total: 25.6s	remaining: 8.5s
1750:	learn: 0.6415471	total: 29.1s	remaining: 4.13s
1999:	learn: 0.6080237	total: 32.3s	remaining: 0us
0:	learn: 1.5979709	total: 18.8ms	remaining: 37.7s
250:	learn: 0.8533128	total: 2.94s	remaining: 20.5s
500:	learn: 0.7486293	total: 5.84s	remaining: 17.5s
750:	learn: 0.6987273	total: 8.63s	remaining: 14.3s
1000:	learn: 0.6603219	total: 12.5s	remaining: 12.5s
1250:	learn: 0.6220031	total: 16.8s	remaining: 10s
1500:	learn: 0.5877514	total: 20.1s	remaining: 6.69s
1750:	learn: 0.5554261	total: 23.6s	remaining: 3.36s
1999:	learn: 0.5278172	total: 26.9s	remaining: 0us
0:	learn: 1.6004096	total: 24.6ms	remaining: 49.3s
250:	learn: 0

In [28]:
# FOLDS = 4
#
# ada_pool = Pool(X, y["Adaptability"])
#
# scores_ada = cv(ada_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)
#
# sys_pool = Pool(X, y["Systemic thinking"])
#
# scores_sys = cv(sys_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)
#
# ana_pool = Pool(X, y["Analytical thinking"])
#
# scores_ana = cv(ana_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)
#
# foc_pool = Pool(X, y["Focus"])
#
# scores_foc = cv(foc_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)

### Inference

In [30]:
subs = pd.read_csv("sample_solution.csv")

In [31]:
list_id = subs["id"].unique().tolist()

df_user_test = df_user[df_user["user_id"].isin(list_id)]
df_user_test

,user_id,period,decision_id
1,10318,3,203
6,10617,3,329
9,10316,3,26
13,10850,4,255
17,10754,3,407
...,...,...,...
60214,10630,4,26
60216,10848,2,409
60221,10054,1,343
60223,10312,3,329


In [32]:
new_datas = create_decision(df_user_test)

In [33]:
test = subs.copy()

for data in new_datas:
    test = test.merge(data, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)


In [34]:
test_w_vec = test.merge(tfidf_features.reset_index(), how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)

In [35]:
# test_w_vec = test.merge(per1, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# test_w_vec = test_w_vec.merge(per2, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# test_w_vec = test_w_vec.merge(per3, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# test_w_vec = test_w_vec.merge(per4, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)

In [36]:
test_vec_scores = test_w_vec.merge(place_score, how="left", left_on="id", right_on="user_id")
test_vec_scores

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,cnt_1,cnt_2,cnt_3,cnt_4,max_1,...,5_per_2_score,5_per_3_score,5_per_4_score,6_per_0_score,6_per_1_score,6_per_2_score,6_per_3_score,6_per_4_score,game_id,team_id
0,10199,0,0,0,0,2.0,2.0,3.0,2.0,386.0,...,16,10,158,55,32,30,30,4,262,1912
1,10539,0,0,0,0,1.0,9.0,15.0,17.0,412.0,...,16,13,158,50,19,34,29,4,204,1458
2,10174,0,0,0,0,NaN,28.0,31.0,20.0,NaN,...,16,45,158,90,29,31,60,4,190,1348
3,10465,0,0,0,0,15.0,18.0,32.0,18.0,410.0,...,16,4,158,67,26,32,24,4,244,1760
4,10066,0,0,0,0,8.0,23.0,16.0,6.0,331.0,...,16,-21,158,50,28,25,22,4,183,1260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,10433,0,0,0,0,18.0,7.0,1.0,NaN,344.0,...,16,0,158,93,61,39,37,4,214,1562
289,10893,0,0,0,0,21.0,52.0,34.0,20.0,418.0,...,16,14,158,96,43,30,32,4,184,1278
290,10909,0,0,0,0,NaN,NaN,NaN,1.0,NaN,...,16,11,158,69,30,34,32,4,206,1474
291,10889,0,0,0,0,6.0,2.0,5.0,1.0,315.0,...,16,34,158,106,41,13,41,4,184,1268


In [37]:
tranformed_data = imp_mean.transform(test_vec_scores)

test_vec_scores_nans = pd.DataFrame(tranformed_data, columns=train_vec_scores.columns)

In [38]:
X = test_vec_scores_nans.drop(targets, axis = 1).drop(["id", "user_id"], axis = 1)
y = test[targets]

y_test_ada = y[["Adaptability"]]

y_test_sys = y[["Systemic thinking"]]

y_test_ana = y[["Analytical thinking"]]

y_test_foc = y[["Focus"]]

In [39]:
pred_ada = model_ada.predict(X)
pred_sys = model_sys.predict(X)
pred_ana = model_ana.predict(X)
pred_foc = model_foc.predict(X)



In [40]:
ans = subs.copy()
#
# for i, col in enumerate(targets):
#     ans[col] = test_pred[:, i]

ans["Adaptability"] = pred_ada
ans["Systemic thinking"] = pred_sys
ans["Analytical thinking"] = pred_ana
ans["Focus"] = pred_foc

In [42]:
ans.to_csv("ans8.csv", index=False)

In [313]:
ans

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,1202,4.0,4.0,5.0,4.0
1,1959,4.0,4.0,4.0,4.0
2,1142,4.0,4.0,5.0,4.0
3,1810,4.0,4.0,4.0,4.0
4,740,4.0,4.0,4.0,4.0
...,...,...,...,...,...
288,1644,4.0,4.0,4.0,4.0
289,2460,5.0,4.0,5.0,5.0
290,2484,4.0,4.0,5.0,5.0
291,2453,5.0,4.0,5.0,5.0
